In [16]:
import pandas as pd
from pymongo import MongoClient
from tabulate import tabulate
import textwrap

# MongoDB connection URI (replace with your actual URI)
uri = "mongodb://localhost:27017"

# Specify the database and collection
database_name = "NewsAnalytics"
collection_name = "RawNews_Hindu"

# Path to the Excel file
excel_file_path = r'C:\Users\91908\Documents\Raja\Share market\Analysis\Trendlyne\Data\Scrip\Scrip.xlsx'

try:
    # Create a new client and connect to the MongoDB server
    with MongoClient(uri) as client:
        # Access the specified collection
        collection = client[database_name][collection_name]
        
        # Query all records in the collection
        all_records = collection.find()
        
        # Store MongoDB documents in a list
        mongo_documents = list(all_records)

        # Read the "Portfolio" sheet from the Excel file
        portfolio_df = pd.read_excel(excel_file_path, sheet_name='Portfolio', header=0)
        
        # Assuming the column containing stock names is named "Portfolio"
        stock_names_in_portfolio = portfolio_df['Portfolio'].tolist()

        # Create a list to store tuples of (Scrip, paragraph_content, label, confidence)
        result_list = []

        # Check if each stock name in the portfolio is present in MongoDB documents
        for document in mongo_documents:
            for stock_name in stock_names_in_portfolio:
                if stock_name.lower() in document['paragraph_content'].lower():
                    # Wrap paragraph content to a maximum of five words per line
                    wrapped_content = '\n'.join(textwrap.wrap(document['paragraph_content'], width=175))
                    result_list.append((
                        stock_name,
                        wrapped_content,
                        document['sentiment']['label'],
                        document['sentiment']['confidence']
                    ))
                    break  # Break if at least one match is found for the current document

        # Create a DataFrame from the result list
        result_df = pd.DataFrame(result_list, columns=['Scrip', 'Paragraph_content', 'Score', 'Confidence'])

        # Display the resulting DataFrame in a nice tabular format
        print(tabulate(result_df, headers='keys', tablefmt='pretty'))

except Exception as e:
    print(e)


+---+--------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+---------------------+
|   | Scrip  |                                                                                Paragraph_content                                                                                |  Score  |     Confidence      |
+---+--------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+---------------------+
| 0 | buying |  Dr. V K Vijayakumar, Chief Investment Strategist at Geojit Financial Services. “Market has turned highly volatile with profit booking triggered by high valuations. Even DIIs  | 4 stars | 0.29071471095085144 |
|   |        |  who have been consistent buyers are booking profits. Buying on dips also is happenin

In [21]:
import pandas as pd
from pymongo import MongoClient
from tabulate import tabulate
import textwrap

# MongoDB connection URI (replace with your actual URI)
uri = "mongodb://localhost:27017"

# Specify the database and collection
database_name = "NewsAnalytics"
collection_name = "RawNews_Hindu"

# Path to the Excel file
excel_file_path = r'C:\Users\91908\Documents\Raja\Share market\Analysis\Trendlyne\Data\Scrip\Scrip.xlsx'

try:
    # Create a new client and connect to the MongoDB server
    with MongoClient(uri) as client:
        # Access the specified collection
        collection = client[database_name][collection_name]
        
        # Query all records in the collection
        all_records = collection.find()
        
        # Store MongoDB documents in a list
        mongo_documents = list(all_records)

        # Read the "Portfolio" sheet from the Excel file
        portfolio_df = pd.read_excel(excel_file_path, sheet_name='Portfolio', header=0)
        
        # Assuming the column containing stock names is named "Portfolio"
        stock_names_in_portfolio = portfolio_df['Portfolio'].tolist()

        # Create a list to store tuples of (Scrip, paragraph_content, label, confidence, date)
        result_list = []

        # Check if each stock name in the portfolio is present in MongoDB documents
        for document in mongo_documents:
            for stock_name in stock_names_in_portfolio:
                if stock_name.lower() in document['paragraph_content'].lower():
                    # Wrap paragraph content to a maximum of five words per line
                    wrapped_content = '\n'.join(textwrap.wrap(document['paragraph_content'], width=170))
                    result_list.append((
                        stock_name,
                        wrapped_content,
                        document['sentiment']['label'],
                        document['sentiment']['confidence'],
                        document['created_at'].strftime('%Y-%m-%d')  # Extract date from created_at
                    ))
                    break  # Break if at least one match is found for the current document

        # Create a DataFrame from the result list
        result_df = pd.DataFrame(result_list, columns=['Scrip', 'Paragraph_content', 'Score', 'Confidence', 'Date'])

        # Display the resulting DataFrame in a nice tabular format
        print(tabulate(result_df, headers='keys', tablefmt='pretty'))

except Exception as e:
    print(e)


+----+--------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+---------------------+------------+
|    | Scrip  |                                                                             Paragraph_content                                                                              |  Score  |     Confidence      |    Date    |
+----+--------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+---------------------+------------+
| 0  | pharma |  GlaxoSmithKline Pharmaceuticalsinformed the exchanges that the Delhi High Court ruled in favour of the company. It has quashed and set aside the demand of the National   | 1 star  | 0.7317842841148376  | 2024-01-03 |
|    |        |  Pharmaceutical Pricing Authority (NPPA) raised 

In [ ]:
Last Good Working : All NSE 1000 stocks

In [30]:
import pandas as pd
from pymongo import MongoClient
from tabulate import tabulate
import textwrap

# MongoDB connection URI (replace with your actual URI)
uri = "mongodb://localhost:27017"

# Specify the database and collection
database_name = "NewsAnalytics"
collection_name = "RawNews_Hindu"

# Path to the Excel file
excel_file_path = r'C:\Users\91908\Documents\Raja\Share market\Analysis\Trendlyne\Data\Scrip\Scrip.xlsx'

try:
    # Create a new client and connect to the MongoDB server
    with MongoClient(uri) as client:
        # Access the specified collection
        collection = client[database_name][collection_name]
        
        # Query all records in the collection
        all_records = collection.find()
        
        # Store MongoDB documents in a list
        mongo_documents = list(all_records)

        # Read the "Portfolio" sheet from the Excel file
        portfolio_df = pd.read_excel(excel_file_path, sheet_name='Top1000NSE', header=0)
        
        # Assuming the column containing stock names is named "Portfolio"
        stock_names_in_portfolio = portfolio_df['Top1000NSE'].tolist()

        # Create a list to store tuples of (Scrip, paragraph_content, label, confidence, date)
        result_list = []

        # Check if each stock name in the portfolio is present in MongoDB documents
        for document in mongo_documents:
            for stock_name in stock_names_in_portfolio:
                if stock_name.lower() in document['paragraph_content'].lower():
                    # Wrap paragraph content to a maximum of five words per line
                    wrapped_content = '\n'.join(textwrap.wrap(document['paragraph_content'], width=140))
                    result_list.append((
                        stock_name,
                        wrapped_content,
                        document['sentiment']['label'],
                        document['sentiment']['confidence'],
                        document['created_at'].strftime('%Y-%m-%d')  # Extract date from created_at
                    ))
                    break  # Break if at least one match is found for the current document

        # Create a DataFrame from the result list
        result_df = pd.DataFrame(result_list, columns=['Scrip', 'Paragraph_content', 'Score', 'Confidence', 'Date'])

        # Sort the DataFrame based on multiple columns in ascending order
        result_df = result_df.sort_values(by=['Date', 'Scrip', 'Score', 'Confidence'], ascending=[False, True, False, False])

        # Display the resulting DataFrame in a nice tabular format
        print(tabulate(result_df, headers='keys', tablefmt='pretty'))

except Exception as e:
    print(e)


+-----+---------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------+---------+---------------------+------------+
|     |              Scrip              |                                                                 Paragraph_content                                                                 |  Score  |     Confidence      |    Date    |
+-----+---------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------+---------+---------------------+------------+
| 207 |            63 Moons             |     Thetransition to new softwareby the country’s largest commodity exchangeMCXand a one-year extension of the ban on derivatives trading in      | 5 stars | 0.38435444235801697 | 2024-01-04 |
|     |                                 |        seven key a

In [ ]:
Last Good Working : only my portfolio

In [33]:
import pandas as pd
from pymongo import MongoClient
from tabulate import tabulate
import textwrap

# MongoDB connection URI (replace with your actual URI)
uri = "mongodb://localhost:27017"

# Specify the database and collection
database_name = "NewsAnalytics"
collection_name = "RawNews_Hindu"

# Path to the Excel file
excel_file_path = r'C:\Users\91908\Documents\Raja\Share market\Analysis\Trendlyne\Data\Scrip\Scrip.xlsx'

try:
    # Create a new client and connect to the MongoDB server
    with MongoClient(uri) as client:
        # Access the specified collection
        collection = client[database_name][collection_name]
        
        # Query all records in the collection
        all_records = collection.find()
        
        # Store MongoDB documents in a list
        mongo_documents = list(all_records)

        # Read the "Portfolio" sheet from the Excel file
        portfolio_df = pd.read_excel(excel_file_path, sheet_name='Portfolio', header=0)
        
        # Assuming the column containing stock names is named "Portfolio"
        stock_names_in_portfolio = portfolio_df['Portfolio'].tolist()

        # Create a list to store tuples of (Scrip, paragraph_content, label, confidence, date)
        result_list = []

        # Check if each stock name in the portfolio is present in MongoDB documents
        for document in mongo_documents:
            for stock_name in stock_names_in_portfolio:
                if stock_name.lower() in document['paragraph_content'].lower():
                    # Wrap paragraph content to a maximum of five words per line
                    wrapped_content = '\n'.join(textwrap.wrap(document['paragraph_content'], width=140))
                    result_list.append((
                        stock_name,
                        wrapped_content,
                        document['sentiment']['label'],
                        document['sentiment']['confidence'],
                        document['created_at'].strftime('%Y-%m-%d')  # Extract date from created_at
                    ))
                    break  # Break if at least one match is found for the current document

        # Create a DataFrame from the result list
        result_df = pd.DataFrame(result_list, columns=['Scrip', 'Paragraph_content', 'Score', 'Confidence', 'Date'])

        # Sort the DataFrame based on multiple columns in ascending order
        result_df = result_df.sort_values(by=['Date', 'Scrip', 'Score', 'Confidence'], ascending=[False, True, False, False])

        # Display the resulting DataFrame in a nice tabular format
        print(tabulate(result_df, headers='keys', tablefmt='pretty'))

except Exception as e:
    print(e)


+----+---------+----------------------------------------------------------------------------------------------------------------------------------------------+---------+---------------------+------------+
|    |  Scrip  |                                                              Paragraph_content                                                               |  Score  |     Confidence      |    Date    |
+----+---------+----------------------------------------------------------------------------------------------------------------------------------------------+---------+---------------------+------------+
| 12 | UJJIVAN | Buzzing Stocks: LIC, Vedanta, Religare, NHPC, BASF India, Airan, Bajaj Finance, IndusInd Bank, Ujjivan SFB, Bharat Forge, SAIL, L&T, Torrent | 5 stars | 0.3067704141139984  | 2024-01-04 |
|    |         |                                                              Power, BGR Energy                                                               |         |           

In [ ]:
Last Good Working : postive and negative words

In [34]:
import pandas as pd
from pymongo import MongoClient
from tabulate import tabulate
import textwrap

# MongoDB connection URI (replace with your actual URI)
uri = "mongodb://localhost:27017"

# Specify the database and collection
database_name = "NewsAnalytics"
collection_name = "RawNews_Hindu"

# Path to the Excel file
excel_file_path = r'C:\Users\91908\Documents\Raja\Share market\Analysis\Trendlyne\Data\Scrip\Scrip.xlsx'

try:
    # Create a new client and connect to the MongoDB server
    with MongoClient(uri) as client:
        # Access the specified collection
        collection = client[database_name][collection_name]
        
        # Query all records in the collection
        all_records = collection.find()
        
        # Store MongoDB documents in a list
        mongo_documents = list(all_records)

        # Read the "Portfolio" sheet from the Excel file
        portfolio_df = pd.read_excel(excel_file_path, sheet_name='PositiveNegative', header=0)
        
        # Assuming the column containing stock names is named "Portfolio"
        stock_names_in_portfolio = portfolio_df['PositiveNegative'].tolist()

        # Create a list to store tuples of (Scrip, paragraph_content, label, confidence, date)
        result_list = []

        # Check if each stock name in the portfolio is present in MongoDB documents
        for document in mongo_documents:
            for stock_name in stock_names_in_portfolio:
                if stock_name.lower() in document['paragraph_content'].lower():
                    # Wrap paragraph content to a maximum of five words per line
                    wrapped_content = '\n'.join(textwrap.wrap(document['paragraph_content'], width=140))
                    result_list.append((
                        stock_name,
                        wrapped_content,
                        document['sentiment']['label'],
                        document['sentiment']['confidence'],
                        document['created_at'].strftime('%Y-%m-%d')  # Extract date from created_at
                    ))
                    break  # Break if at least one match is found for the current document

        # Create a DataFrame from the result list
        result_df = pd.DataFrame(result_list, columns=['Scrip', 'Paragraph_content', 'Score', 'Confidence', 'Date'])

        # Sort the DataFrame based on multiple columns in ascending order
        result_df = result_df.sort_values(by=['Date', 'Scrip', 'Score', 'Confidence'], ascending=[False, True, False, False])

        # Display the resulting DataFrame in a nice tabular format
        print(tabulate(result_df, headers='keys', tablefmt='pretty'))

except Exception as e:
    print(e)


+-----+-------------+----------------------------------------------------------------------------------------------------------------------------------------------+---------+---------------------+------------+
|     |    Scrip    |                                                              Paragraph_content                                                               |  Score  |     Confidence      |    Date    |
+-----+-------------+----------------------------------------------------------------------------------------------------------------------------------------------+---------+---------------------+------------+
| 110 |     Buy     |  JP Morgan on Tata Motors: Upgrade to Overweight on Company, raise target price at Rs 925/sh JP Morgan on Infosys: Upgrade to Overweight on  | 5 stars | 0.3712768852710724  | 2024-01-04 |
|     |             | Company, raise target price at Rs 1800/sh JP Morgan on LTTS: Upgrade to Overweight on Company, raise target price at Rs 5800/sh JP Morgan 